# 3 함수


---

## 19 함수가 여러 값을 반환하는 경우, 절대 네 값 이상을 언패킹하지 말라

언패킹 구문의 한 효과는 함수가 둘 이상의 값을 반환할 수 있다는 점이다. 아래 예제는 악어 개체군과 관련해서 통계를 계산하는 코드다. 개체군에서 가장 긴 악어와 가장 짧은 악어의 신장을 계산할 것이다.

In [1]:
def get_stats(numbers):
    minimum = min(numbers)
    maximum = max(numbers)
    return minimum, maximum

lengths = [63, 73, 72, 60, 67, 66, 71, 61, 72, 70]

minimum, maximum = get_stats(lengths)    # minimum, maximum 두 값이 반환된다.

print(f'최소: {minimum}, 최대: {maximum}')

최소: 60, 최대: 73


별표 식을 사용해서 여러 값을 반환 받을 수도 있다. 악어 개체군의 신장이 평균에 비해 얼마나 큰지 계산하는 함수가 있다고 가정하자. 여기에 중간 값을 반환 받는 부분을 추가해 보자.

In [2]:
def get_avg_ratio(numbers):
    average = sum(numbers) / len(numbers)
    scaled = [x / average for x in numbers]
    scaled.sort(reverse=True)
    return scaled

longest, *middle, shortest = get_avg_ratio(lengths)

print(f'최대 길이: {longest:>4.0%}')
print(f'최소 길이: {shortest:>4.0%}')

최대 길이: 108%
최소 길이:  89%


여기서 몸 길이의 평균, 중앙값, 악어 개체군의 개체 수까지 반환하게 바꾸면 어떨까? 함수를 변경해 보자.

In [3]:
def get_stats(numbers):
    minimum = min(numbers)
    maximum = max(numbers)
    count = len(numbers)              # 악어 개체 수
    average = sum(numbers) / count    # 평균 값

    sorted_numbers = sorted(numbers)
    middle = count // 2
    # 중앙값
    if count % 2 == 0:
        lower = sorted_numbers[middle - 1]
        upper = sorted_numbers[middle]
        median = (lower + upper) / 2
    else:
        median = sorted_numbers[middle]

    return minimum, maximum, average, median, count

minimum, maximum, average, median, count = get_stats(lengths)

print(f'최소 길이: {minimum}, 최대 길이: {maximum}')
print(f'평균: {average}, 중앙값: {median}, 개수: {count}')

최소 길이: 60, 최대 길이: 73
평균: 67.5, 중앙값: 68.5, 개수: 10


이 코드에는 두 가지 문제가 있다. 먼저 첫 번째 문제는 모든 return 값이 수(number)이기 때문에 순서를 착각하기 쉽다.(예를 들어 중앙값과 평균이 바뀔 수 있다.)

In [ ]:
# 올바른 사용
minimum, maximum, average, median, count = get_stats(lengths)

# 순서를 착각한 사용
minimum, maximum, median, average, count = get_stats(lengths)

두 번째 문제는 함수를 호출하는 부분이나, return을 언패킹하는 부분이 너무 길다는 것이다. 따라서 가독성이 떨어진다.

이런 문제를 피하려면 여러 값을 return할 때나, 언패킹할 때 변수를 네 개 이상 사용하지 말아야 한다.(세 개까지만 쓰자.) 이보다 많은 값을 언패킹해야 하는 상황이면 경량 클래스(lightweight class)나 namedtuple을 사용하고, 함수도 이런 값을 반환하게 만드는 편이 낫다.

<br/>

## 20 None을 반환하기보다는 예외를 발생시켜라

유틸리티 함수를 만들면서 None을 반환하며 이 값에 특별한 의미를 부여하려는 경우가 있다. 경우에 따라서는 타당해 보인다.

아래 예제는 나누기를 수행하는 함수로, 0으로 나누게 될 경우 None을 반환하게 만들었다.

In [4]:
def careful_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None

이렇게 설계했을 경우 다음과 같이 사용할 수 있다.

In [5]:
x, y = 1, 0
result = careful_divide(x, y)
if result is None:
    print('잘못된 입력')

잘못된 입력


그런데 이렇게 설계를 했다면, 실수로 None이 아닌 False로 검사를 시행할 수 있다. 그렇게 되면 0이나 빈 값을 받아서 의도와 달라질 수 있다.

In [6]:
x, y = 0, 5
result = careful_divide(x, y)
if not result:
    print('잘못된 입력')

잘못된 입력


이런 실수를 저지르지 않기 위한 두 가지 해결책이 있다.

첫 번째 방법은 반환 값을 2-튜플로 분리하는 것이다. 이 튜플의 첫 번째 부분은 연산이 성공인지 실패인지 표시한다. 두 번째 부분은 (계산에 성공하면) 실제 결과값을 저장한다.

In [7]:
def careful_divide(a, b):
    try:
        return True, a / b
    except ZeroDivisionError:
        return False, None

이 함수를 호출하는 쪽에서는 튜플을 언패킹해야 한다. 항상 연산이 성공했는지를 살피게 된다.

In [ ]:
success, result = careful_divide(x, y)
if not success:
    print('잘못된 입력')

다만 이 방법대로 해도 실수할 가능성이 높다. 호출하는 쪽에서 튜플의 첫 번째 부분을 쉽게 무시(변수를 사용하지 않고 밑줄로 무시)할 수 있기 때문이다.

In [ ]:
_, result = careful_divide(x, y)
if not success:
    print('잘못된 입력')

이런 실수를 줄일 더 나은 두 번째 방법은, 결코 None을 반환하지 않는 것이다. 대신 Exception을 발생시킨다.

In [8]:
def careful_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('잘못된 입력')

이러면 호출자는 더 이상 조건문으로 반환 값을 검사하지 않아도 된다. 대신에 return 값이 항상 올바르다고 가정하고, try 문의 else 블록에서 이 값을 즉시 사용할 수 있다.

In [9]:
x, y = 5, 2
try:
    result = careful_divide(x, y)
except ValueError:
    print('잘못된 입력')
else:
    print('결과는 %.1f 입니다.' % result)

결과는 2.5 입니다.


마찬가지의 방법을 타입 애너테이션을 사용하는 코드에도 적용할 수 있다.

In [10]:
def careful_divide(a: float, b: float) -> float:
    """a를 b로 나눈다.
    Raise:
        ValueError: b가 0이어서 나눗셈을 할 수 없을 때
    """
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('잘못된 입력')